In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, MatchAny, FieldCondition, Filter, Prefetch, FusionQuery

import pandas as pd
import numpy as np
import openai
import json
import tiktoken

### Retrieve all item IDs from Amazon Items Qdrant Collection


In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")


In [3]:
dummy_vector = np.zeros(1536).tolist()


In [4]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [5]:
payload.points


[ScoredPoint(id=219, version=2, score=0.0, payload={'parent_asin': 'B0C9YG3BK4'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=346, version=2, score=0.0, payload={'parent_asin': 'B0C2Q8BDTX'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=360, version=2, score=0.0, payload={'parent_asin': 'B09Z884J22'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=104, version=2, score=0.0, payload={'parent_asin': 'B0BXKHLT54'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=134, version=2, score=0.0, payload={'parent_asin': 'B09TFS9298'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=113, version=2, score=0.0, payload={'parent_asin': 'B08ZK5X1FN'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=136, version=2, score=0.0, payload={'parent_asin': 'B0C8TK3VV3'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=333, version=2, score=0.0, payload={'parent_asin': 'B09V2TMHJD'}, vector=N

In [6]:
parent_asin_list = [item.payload["parent_asin"] for item in payload.points]


In [7]:
parent_asin_list

['B0C9YG3BK4',
 'B0C2Q8BDTX',
 'B09Z884J22',
 'B0BXKHLT54',
 'B09TFS9298',
 'B08ZK5X1FN',
 'B0C8TK3VV3',
 'B09V2TMHJD',
 'B0B2RKB1HP',
 'B0B38P75Q1',
 'B09VY6CVH3',
 'B0C8CNCPT3',
 'B09RGT465X',
 'B09YGFTHZS',
 'B0BX2RDHB8',
 'B09QKQR55N',
 'B09SZCFJGD',
 'B09Y5TLKGC',
 'B0C8P217BG',
 'B0BXGZ9GNM',
 'B09Y1S3668',
 'B0BCP8SPY5',
 'B0BHNSFT7V',
 'B0BNX17SHQ',
 'B0BRKVLTP5',
 'B09RSRH7MP',
 'B0BZ41HJ66',
 'B0B6YVKN13',
 'B0B832GTH6',
 'B0B87KVMTZ',
 'B0BHWJKLKH',
 'B0BLVYKJSX',
 'B0C1RSJ3WC',
 'B09PYK37K4',
 'B0BX7JKQFH',
 'B0BGXKQ4MP',
 'B09VG8JBNM',
 'B0BDF7RPR7',
 'B0B852V1ZT',
 'B0CC8QJ859',
 'B09YR9DG2P',
 'B0BZSGNYXM',
 'B0BKG27JX5',
 'B0BBLJ352Q',
 'B0BGGG3S1M',
 'B0C49XLZW1',
 'B0BYVW51MK',
 'B0BK8458S2',
 'B09LCXK4JY',
 'B0B19SL9TF',
 'B0C2C7NMGP',
 'B0B8XB4WPB',
 'B0C5JG82J7',
 'B0BMTKDPPN',
 'B0BGPSZMM5',
 'B0BYJLC7YQ',
 'B0C2SXTFZ1',
 'B0CDV64HL4',
 'B0C61SWSG9',
 'B09VC6NL2Y',
 'B0C7WPSKY1',
 'B09Q3GT1T2',
 'B0BQJ9MFD3',
 'B09PRRB7QX',
 'B0B51L5HTD',
 'B0C4KD57CC',
 'B0BK3LYP

In [8]:
len(parent_asin_list)


500

### Load Amazon Reviews Dataset


In [9]:
df_reviews = pd.read_json("../../data/Electornics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)


In [10]:
# Count how many times each (parent_asin, user_id) pair appears
duplicate_counts = df_reviews.groupby(['parent_asin', 'user_id']).size()
duplicate_counts = duplicate_counts[duplicate_counts > 1].sort_values(ascending=False)

print(f"Number of duplicate (parent_asin, user_id) pairs: {len(duplicate_counts)}")
duplicate_counts.head(10)  # Show top 10 most duplicated
#To remove duplicates if found:
# Keep first occurrence
df_reviews_clean = df_reviews.drop_duplicates(subset=['parent_asin', 'user_id'], keep='first')

print(f"Original: {len(df_reviews)}, After dedup: {len(df_reviews_clean)}")

Number of duplicate (parent_asin, user_id) pairs: 96602
Original: 299618, After dedup: 193430


In [11]:
df_reviews = df_reviews_clean


In [12]:
df_reviews

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Perfect!,This is perfect! Thank you so much!!! I absolu...,[],B09992M2LX,B09ZPV8WBV,AHX4XWVVQUKT3FCNWCVASDF4Q56Q,2022-08-05 04:06:39.589,0,True
1,5,3ft mini usb cables,I don't have many things that still use a mini...,[],B09Y94B2NM,B09Y95BMKX,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-16 16:03:28.714,3,True
2,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True
3,5,Great to Have Around,My husband and I were recently working a booth...,[],B09M89JN7B,B0BYYGZHG5,AHV6QCNBJNSGLATP56JAWJ3C4G2A,2022-03-22 01:43:49.342,0,False
4,5,Easy to use,Work as advertised and at a very good price.,[],B07T55DL33,B0B2JWCMCY,AG7WKTZINOFIXMZJYIPKIB7PV7NQ,2019-12-28 06:12:24.960,0,True
...,...,...,...,...,...,...,...,...,...,...
202762,5,Great Size for Travel!,Good size meaning it is very portable and I wo...,[{'small_image_url': 'https://images-na.ssl-im...,B085HMC4G5,B0CC1H1WFT,AFLE5TJJJER6OCJB7O4M4667V5MA,2020-07-13 19:07:48.918,0,True
202764,5,Great product,"Great picture, great 2 way communication, and ...",[],B0B3RMQGKP,B0C8CNCPT3,AH2BVQ7AKGQTASJ6TX3YNSLBQUXQ,2023-06-07 21:12:30.951,0,True
202765,5,Very easy to pair and install,Very easy to install. Great picture quality. ...,[],B0B3RMQGKP,B0C8CNCPT3,AHJCPX3AGNBIKDHRJ3SYJNWHHLKQ,2023-03-23 22:36:49.650,0,True
202766,5,Awesome value and look. Blends well with high ...,This cable box isn't cheap plastic constructio...,[],B07QCHML1Y,B0BLY789SC,AFFZP5WQGDTWFCP3Q4TCAMJWWC6Q,2019-08-13 14:27:51.153,0,True


In [13]:
len(df_reviews)

193430

In [14]:
# len(df_reviews)


In [15]:
df_reviews_sample = df_reviews[df_reviews["parent_asin"].isin(parent_asin_list)]


In [16]:
len(df_reviews_sample)


59336

In [17]:
# len(df_reviews_sample)


### Define functions to preprocess reviews data


In [18]:
def preprocess_reviews_data(row):
    return f"{row['title']} {row['text']}"

In [19]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")


In [20]:
encoding.encode("Can I get some earphones?")


[6854, 358, 636, 1063, 2487, 17144, 30]

In [21]:
def token_count(row, model="text-embedding-3-small"):

    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(row["preprocessed_data"]))

In [22]:
df_reviews_sample["preprocessed_data"] = df_reviews_sample.apply(preprocess_reviews_data, axis=1)


/var/folders/7d/0kh1mw2n62x6t5x8lp0hgjt80000gp/T/ipykernel_1336/1283496592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_sample["preprocessed_data"] = df_reviews_sample.apply(preprocess_reviews_data, axis=1)


In [23]:
df_reviews_sample["preprocessed_data_token_count"] = df_reviews_sample.apply(token_count, axis=1)


/var/folders/7d/0kh1mw2n62x6t5x8lp0hgjt80000gp/T/ipykernel_1336/1408307758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_sample["preprocessed_data_token_count"] = df_reviews_sample.apply(token_count, axis=1)


In [24]:
df_reviews_sample.head()


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,preprocessed_data,preprocessed_data_token_count
68,5,Great for velcroing LARGE things!,"First, note that this isn't just a hoop-and-lo...",[],B09QSY9B7J,B09QY7YYBJ,AGDSEYGSA5K664EUHWKV3ARDXO2Q,2022-03-03 18:48:36.284,0,False,"Great for velcroing LARGE things! First, note ...",154
140,5,Huge,Love it. Big and wide. Great clarity. I use it...,[],B09ZYTFX33,B0BMQP624X,AFOHEEK7Z7O37JDYNEMCZSBUZHYA,2022-09-22 20:52:44.420,1,True,Huge Love it. Big and wide. Great clarity. I u...,30
207,5,work as needed,Solid brand. Chose these to hold hubs CPap mac...,[],B09QT1Y48G,B09QY7YYBJ,AEF4QCBYMXQN3RDFNM4ZXM67UTJA,2022-03-17 12:48:36.709,0,False,work as needed Solid brand. Chose these to hol...,53
208,5,"Great noise cancellation and bass, clear sound...",These headphones offer crisp sound with deep b...,[{'small_image_url': 'https://m.media-amazon.c...,B0B1WYQHLZ,B0B8NCR19L,AH5HSMUMRJRDWBPFDQ6G3EAAOSFQ,2022-07-09 20:35:33.477,0,False,"Great noise cancellation and bass, clear sound...",147
236,5,works great.,Used this for getting analog audio out of a hy...,[],B00KBQZC4M,B08NW1RXPF,AHNN4OIGZQZERZQ5XMXNKPIC7MLA,2016-07-01 13:55:24.000,0,True,works great. Used this for getting analog audi...,18


In [25]:
len(df_reviews_sample)


59336

In [26]:
df_reviews_sample = df_reviews_sample[df_reviews_sample["preprocessed_data_token_count"] < 8192]


In [27]:
len(df_reviews_sample)


59336

In [28]:
total_tokens = df_reviews_sample["preprocessed_data_token_count"].sum()


In [29]:
total_tokens

np.int64(3593955)

### Create a new Qdrant collection for reviews


In [31]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-01-reviews",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [32]:
qdrant_client.create_payload_index(
    collection_name="Amazon-items-collection-01-reviews",
    field_name="parent_asin",
    field_schema=PayloadSchemaType.KEYWORD
)


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Embedding functions


In [33]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

    

In [34]:
from tqdm import tqdm

def get_embedding_batch(text_list: list[str], model: str = "text-embedding-3-small", batch_size: int = 100) -> list[list[float]]:
    """Get embeddings for a list of texts in batches."""
    all_embeddings = []
    
    for i in tqdm(range(0, len(text_list), batch_size), desc="Embedding"):
        batch = text_list[i:i + batch_size]
        response = openai.embeddings.create(input=batch, model=model)
        all_embeddings.extend([e.embedding for e in response.data])
    
    return all_embeddings

### Embed the text and add additional fields to the payload for each vector for reviews

In [35]:
data_to_embed_reviews = df_reviews_sample[["preprocessed_data","parent_asin"]].to_dict(orient="records")

In [36]:
data_to_embed_reviews

[{'preprocessed_data': 'Great for velcroing LARGE things! First, note that this isn\'t just a hoop-and-loop product. This is VELCRO, the brand name itself! So you know the expectations are set high. And they deliver.<br />I\'ve owned similar products that are good for wrapping up cables I bring to jobs, but when I\'m doing a live stream event and need to run hundreds of feet of CAT5 cable, you need a BIG strap to secure them up when wrapped up to go. These 30" straps are not only big, but they are stretchy as well, so I could easily make a bundle of very thick cable and hold it all together like a bow. This is exactly what I wanted and needed, thank you Velcro!',
  'parent_asin': 'B09QY7YYBJ'},
 {'preprocessed_data': 'Huge Love it. Big and wide. Great clarity. I use it for work - email, spreadsheets, applications. Extra space improves productivity.',
  'parent_asin': 'B0BMQP624X'},
 {'preprocessed_data': 'work as needed Solid brand. Chose these to hold hubs CPap machine hose out of the

In [37]:
text_to_embed_reviews = [data["preprocessed_data"] for data in data_to_embed_reviews]

In [38]:
text_to_embed_reviews[0]

'Great for velcroing LARGE things! First, note that this isn\'t just a hoop-and-loop product. This is VELCRO, the brand name itself! So you know the expectations are set high. And they deliver.<br />I\'ve owned similar products that are good for wrapping up cables I bring to jobs, but when I\'m doing a live stream event and need to run hundreds of feet of CAT5 cable, you need a BIG strap to secure them up when wrapped up to go. These 30" straps are not only big, but they are stretchy as well, so I could easily make a bundle of very thick cable and hold it all together like a bow. This is exactly what I wanted and needed, thank you Velcro!'

In [39]:
embedding_reviews = get_embedding_batch(text_to_embed_reviews, batch_size=500)

Embedding:   0%|          | 0/119 [00:00<?, ?it/s]

Embedding: 100%|██████████| 119/119 [03:58<00:00,  2.00s/it]


In [40]:
len(embedding_reviews)

59336

In [41]:
pointstructs = []
i = 1
for embedding, data in zip(embedding_reviews, data_to_embed_reviews):
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload={
                "text": data["preprocessed_data"],
                "parent_asin": data["parent_asin"],
            }
        )
    )
    i += 1

In [42]:
batch_size_qdrant = 100

for i in tqdm(range(0, len(pointstructs), batch_size_qdrant), desc="Upserting to Qdrant"):
    batch = pointstructs[i:i + batch_size_qdrant]
    qdrant_client.upsert(
        collection_name="Amazon-items-collection-01-reviews",
        wait=True,
        points=batch
    )

Upserting to Qdrant: 100%|██████████| 594/594 [01:51<00:00,  5.32it/s]


###

### A function to run search against reviews on a prefiltered set of product IDs

In [43]:
def retrieve_prefiltered_reviews_data(query, parent_asins, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-reviews",
        prefetch=[
            Prefetch(
                query=query_embedding,
                filter=Filter(
                    must=[
                        FieldCondition(
                            key="parent_asin",
                            match=MatchAny(
                                any=parent_asins
                            )
                        )
                    ]
                ),
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k
    )

    return results

In [44]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B08NW1RXPF"])

In [45]:
reviews.points

[ScoredPoint(id=657, version=8, score=0.5, payload={'text': 'One Star Picture quality was horrible.', 'parent_asin': 'B08NW1RXPF'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2122, version=23, score=0.33333334, payload={'text': 'Three Stars it works. quality is trash.', 'parent_asin': 'B08NW1RXPF'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1417, version=16, score=0.25, payload={'text': 'Four Stars Pretty low video quality but to be expected', 'parent_asin': 'B08NW1RXPF'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2500, version=26, score=0.2, payload={'text': "... on and off turns on and off quality looks like a bad VHS tape there are several others out ... Item cuts on and off turns on and off quality looks like a bad VHS tape there are several others out there at half the cost but I'm sure do you just as well", 'parent_asin': 'B08NW1RXPF'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1788, version=19

In [48]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B0BMQP624X", "B09QY7YYBJ"])

In [49]:
reviews.points

[ScoredPoint(id=2193, version=23, score=0.5, payload={'text': 'Garbage Poor quality, absolutely useless.', 'parent_asin': 'B09QY7YYBJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1132, version=13, score=0.33333334, payload={'text': 'Surprised such good reviews The picture quality on this was not horrible but as a designer was not high enough quality to work with. It was grainy and not sharp. Returning.', 'parent_asin': 'B0BMQP624X'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1246, version=14, score=0.25, payload={'text': 'It is not as good as expected I am a graphic designer/digital artist. I am using it as my 3rd screen.<br />The picture quality is not good. You can see the pixels clearly.', 'parent_asin': 'B0BMQP624X'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2089, version=22, score=0.2, payload={'text': 'Horribly made, cheap and fake warranty  ', 'parent_asin': 'B0BMQP624X'}, vector=None, shard_key=None, order_value

In [50]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B0BMQP624X", "B09QY7YYBJ"],k=20)

In [51]:
reviews.points

[ScoredPoint(id=2193, version=23, score=0.5, payload={'text': 'Garbage Poor quality, absolutely useless.', 'parent_asin': 'B09QY7YYBJ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1132, version=13, score=0.33333334, payload={'text': 'Surprised such good reviews The picture quality on this was not horrible but as a designer was not high enough quality to work with. It was grainy and not sharp. Returning.', 'parent_asin': 'B0BMQP624X'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1246, version=14, score=0.25, payload={'text': 'It is not as good as expected I am a graphic designer/digital artist. I am using it as my 3rd screen.<br />The picture quality is not good. You can see the pixels clearly.', 'parent_asin': 'B0BMQP624X'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2089, version=22, score=0.2, payload={'text': 'Horribly made, cheap and fake warranty  ', 'parent_asin': 'B0BMQP624X'}, vector=None, shard_key=None, order_value

In [52]:
for point in reviews.points:
    print(point.payload["parent_asin"])
    print(point.payload["text"])
    print("-"*100)

B09QY7YYBJ
Garbage Poor quality, absolutely useless.
----------------------------------------------------------------------------------------------------
B0BMQP624X
Surprised such good reviews The picture quality on this was not horrible but as a designer was not high enough quality to work with. It was grainy and not sharp. Returning.
----------------------------------------------------------------------------------------------------
B0BMQP624X
It is not as good as expected I am a graphic designer/digital artist. I am using it as my 3rd screen.<br />The picture quality is not good. You can see the pixels clearly.
----------------------------------------------------------------------------------------------------
B0BMQP624X
Horribly made, cheap and fake warranty  
----------------------------------------------------------------------------------------------------
B0BMQP624X
You get what you pay for Decent quality/pixels.<br /><br />The main big issue is the fact that this monitor is sl